Project- Cricket Umpire Hand Detection

In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands and drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize Hand Detector
def initialize_hand_detector():
    return mp_hands.Hands(
        static_image_mode=False,         # False for real-time detection
        max_num_hands=2,                 # Detect up to 2 hands
        min_detection_confidence=0.5,    # Minimum detection confidence
        min_tracking_confidence=0.5      # Minimum tracking confidence
    )

# Draw hand landmarks
def draw_hand_landmarks(frame, landmarks):
    mp_drawing.draw_landmarks(
        frame,
        landmarks,
        mp_hands.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
    )

# Detect "No Ball" signal
def detect_no_ball_signal(landmarks):
    wrist_y = landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    thumb_tip_y = landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_tip_y = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

    if thumb_tip_y < index_tip_y and wrist_y < 0.4:  # Hand raised above head
        return True
    return False

# Detect "Out" signal
def detect_out_signal(landmarks):
    wrist_y = landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    index_tip_y = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip_y = landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y

    if index_tip_y < middle_tip_y and wrist_y < 0.6:  # Index finger raised
        return True
    return False

# Detect "Four Runs" signal
def detect_four_runs_signal(landmarks):
    wrist_y = landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    thumb_tip_y = landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
    index_tip_y = landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y

    if wrist_y < 0.6 and abs(index_tip_y - thumb_tip_y) < 0.1:  # Waving horizontally
        return True
    return False

# Detect "Wide Ball" signal
def detect_wide_ball_signal(left_landmarks, right_landmarks):
    left_wrist_y = left_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    right_wrist_y = right_landmarks.landmark[mp_hands.HandLandmark.WRIST].y

    # left_wrist_y = landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    # right_wrist_y = landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    
    # left_wrist_x = landmarks.landmark[mp_hands.HandLandmark.WRIST].x
    # right_wrist_x = landmarks.landmark[mp_hands.HandLandmark.WRIST].x
     # Wide ball detection (both hands extended horizontally)
    if left_wrist_y >= 0.3 and right_wrist_y >= 0.3:
        if abs(left_wrist_x - right_wrist_x) > 0.2:  # Check horizontal distance
            return True
    return False

# Detect "Six Runs" signal
def detect_six_runs_signal(left_landmarks, right_landmarks):
    left_wrist_y = left_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
    right_wrist_y = right_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
   
    # Six run detection (both hands raised above head)
    if left_wrist_y < 0.3 and right_wrist_y < 0.3:
        return True
    return False

# Main function to run hand detection and signal identification
def detect_signals():
    cap = cv2.VideoCapture(0)
    hands = initialize_hand_detector()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Ignoring empty frame.")
            continue

        # Flip the frame for user experience
        frame = cv2.flip(frame, 1)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            if len(result.multi_hand_landmarks) == 1:
                # One hand detected
                hand_landmarks = result.multi_hand_landmarks[0]
                draw_hand_landmarks(frame, hand_landmarks)

                if detect_no_ball_signal(hand_landmarks):
                    cv2.putText(frame, "No Ball Signal Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                elif detect_out_signal(hand_landmarks):
                    cv2.putText(frame, "Out Signal Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                elif detect_four_runs_signal(hand_landmarks):
                    cv2.putText(frame, "Four Runs Signal Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

            elif len(result.multi_hand_landmarks) == 2:
                # Two hands detected
                left_hand, right_hand = result.multi_hand_landmarks
                draw_hand_landmarks(frame, left_hand)
                draw_hand_landmarks(frame, right_hand)

                if detect_wide_ball_signal(left_hand, right_hand):
                    cv2.putText(frame, "Wide Ball Signal Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                elif detect_six_runs_signal(left_hand, right_hand):
                    cv2.putText(frame, "Six Runs Signal Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)

        cv2.imshow('Hand Signal Detection', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the main function
detect_signals()